<p>
<img src="../imgs/EII-ULPGC-logo.jpeg" width="430px" align="right">

# **NOTEBOOK 8.1**
---

# **Function Calling**

Las function calling son una capacidad de los modelos de lenguaje que les permite identificar cuándo una consulta del usuario requiere ejecutar una función externa, generar automáticamente los argumentos estructurados para esa función y devolverlos en un formato que el modelo los pueda interpretar. De este modo, el modelo no solo produce texto, sino que actúa como un orquestador: decide si usar una herramienta, construye los parámetros adecuados y permite que el sistema ejecute la función real (por ejemplo, consultar una API, acceder a una base de datos o realizar un cálculo), integrando después el resultado en la conversación.

https://platform.openai.com/docs/guides/function-calling

<div align="center">
    <img src="imgs/function-calling-diagram-steps.png" width="500px">
</div>
  
### **Ejemplo básico de Function Calling con la API de OpenAI**

In [ ]:
from openai import OpenAI
import json
import os

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
]

def get_horoscope(sign):
    return f"{sign}: Next Tuesday you will befriend a baby otter."

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
)

# Save function call outputs for subsequent requests
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # 3. Execute the function logic for get_horoscope
            horoscope = get_horoscope(json.loads(item.arguments))
            
            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

print("Final input:")
print(input_list)

response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

Final input:
[{'role': 'user', 'content': 'What is my horoscope? I am an Aquarius.'}, ResponseReasoningItem(id='rs_05de88ec025a636800691f8fdf650881a08d4d0b8860b9d6b0', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"sign":"Aquarius"}', call_id='call_TeW9yCYRkLnuU47ATHljEum9', name='get_horoscope', type='function_call', id='fc_05de88ec025a636800691f8fe39a8c81a081a526c55d7d4090', status='completed'), {'type': 'function_call_output', 'call_id': 'call_TeW9yCYRkLnuU47ATHljEum9', 'output': '{"horoscope": "{\'sign\': \'Aquarius\'}: Next Tuesday you will befriend a baby otter."}'}]
Final output:
{
  "id": "resp_05de88ec025a636800691f8fe474f481a0bcaf7bce6da9d25e",
  "created_at": 1763676132.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond only with a horoscope generated by a tool.",
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "rs_05de88